In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

Создадим датасет с данными о продажах продукции по различным компаниям и рыночной стоимостью этих компаний.
Если упростить один из методов оценки стоимости компании, то стоимость складывается из
1. имеющихся активов (предположим, что все компании располагают активами в $200200),
2. прогноза будущих доходов за какой-то период времени (возьмем 10 лет) за вычетом всех расходов на поддержание бизнеса, налогов и прочих обязательств (будем считать, что на все это уходит 80% от дохода)

In [2]:
# Создаем сэмпл
n_samples = 500

# количество персонала в компании
number_of_stuff = np.random.choice(500, n_samples) + 10
# количество проданных единиц товара за год
quantity_of_sold_goods  = np.random.choice(100100, n_samples) + 1000
# средняя цена единицы товара
average_price_of_sold_goods  = np.random.choice(100, n_samples) + 10

# стоимость компании
company_value = quantity_of_sold_goods * average_price_of_sold_goods * 10 * 0.8 + 200200

data = pd.DataFrame({'number_of_stuff': number_of_stuff, 'quantity_of_sold_goods': quantity_of_sold_goods, 
                     'average_price_of_sold_goods': average_price_of_sold_goods, 'company_value': company_value})
data.head(5)

,number_of_stuff,quantity_of_sold_goods,average_price_of_sold_goods,company_value
0,177,13830,21,2523640.0
1,197,50964,68,27924616.0
2,137,49846,62,24923816.0
3,432,97146,87,67813816.0
4,312,19642,66,10571176.0


In [3]:
# Обучаем модель линейной регрессии и смотрим на получившиеся веса, сдвиг и среднюю абсолютную ошибку

from sklearn.metrics import mean_absolute_error

# Вводим иксы
X = data[['number_of_stuff', 'quantity_of_sold_goods', 'average_price_of_sold_goods']]
# Вводим игреки
y = data['company_value']
# Обучаем модель
reg = LinearRegression().fit(X, y)
# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))
# Сдвиг, который был равен 200200 по условию
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и иксов предсказываем игреки
pred_values = reg.predict(data[['number_of_stuff', 'quantity_of_sold_goods', 'average_price_of_sold_goods']])
# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-3.30675885e+02  4.75301298e+02  4.22208080e+05]
Bias: -24791079.557531487
Error: 5119070.276585985


In [4]:
y.median()

18878804.0

В результате видим достаточно сомнительные веса, неверный сдвиг и огромную среднюю абсолютную ошибку (около четверти).  
Попробуем убрать лишний признак, т.е. количество персонала, которое не влияет на рыночную капитализацию компании (строго говоря, косвенно немного влияет, но это влияние настолько невелико, что можно легко им пренебречь безо всякого ущерба для модели).

In [5]:
# Вводим иксы
X = data[['quantity_of_sold_goods', 'average_price_of_sold_goods']]
# Вводим игреки
y = data['company_value']
# Обучаем модель
reg = LinearRegression().fit(X, y)
# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))
# Сдвиг, который был равен 200200 по условию
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и иксов предсказываем игреки
pred_values = reg.predict(data[['quantity_of_sold_goods', 'average_price_of_sold_goods']])
# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [   475.41285311 422157.06453108]
Bias: -24876771.7922563
Error: 5118724.44052206


Результаты не стали лучше.  
Чтобы понять почему, нужно ответить на вопрос: что из себя представляет линейная модель?  
f(x,b) = b0 + b1x1 + b2x2 и т.д., где b - параметры (коэффициенты) регрессии, х - регрессоры (факторы модели).  
В нашем примере получается так: b0 + b1x1x2, что не является линейной зависимостью и приводит к огромным погрешностям модели.  
Поэтому необходимо создать новый признак, чтобы получить зависимость вида: b0 + b1x1.

In [6]:
# Создаем новый признак - годовая прибыль (annual profit)
data['annual_profit'] = data['quantity_of_sold_goods'] * data['average_price_of_sold_goods']
data.head(5)

,number_of_stuff,quantity_of_sold_goods,average_price_of_sold_goods,company_value,annual_profit
0,177,13830,21,2523640.0,290430
1,197,50964,68,27924616.0,3465552
2,137,49846,62,24923816.0,3090452
3,432,97146,87,67813816.0,8451702
4,312,19642,66,10571176.0,1296372


In [7]:
# Вводим иксы
X = data[['annual_profit']]
# Вводим игреки
y = data['company_value']
# Обучаем модель
reg = LinearRegression().fit(X, y)
# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))
# Сдвиг, который был равен 200200 по условию
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и иксов предсказываем игреки
pred_values = reg.predict(data[['annual_profit']])
# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [8.]
Bias: 200200.0
Error: 0.0


Вуаля! Абсолютно верные вес и сдвиг, нулевая ошибка. И все это по причине того, что мы создали линейную зависимость с помощью нового признака.